CSCI-E89 Deep Learning  
Spring 2019  
Final Project  
Sulkava, Pietari  

In [1]:
%load_ext autoreload
%autoreload 2
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Lambda
from keras.layers import Input
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers import Multiply
from keras.optimizers import Adam
from keras.optimizers import RMSprop
import numpy as np
import gym
from collections import deque
import random
from atari_wrappers import *
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [5]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [29]:
learning_rate = 0.001
gamma = 0.95
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
episodes = 5000
memory = deque(maxlen=1000000)

In [30]:
model = Sequential()
model.add(Dense(24, input_dim=observation_space, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(action_space, activation='linear'))
model.compile(loss='mse', optimizer=Adam(lr=learning_rate))

In [31]:
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

In [32]:
def replay(batch_size):
    # Sample minibatch from the memory
    minibatch = random.sample(memory, batch_size)

    # Extract informations from each memory
    for state, action, reward, next_state, done in minibatch:

        # if done, make our target reward
        target = reward

        if not done:
          # predict the future discounted reward
          target = reward + gamma * \
                   np.amax(model.predict(next_state)[0])

        # make the agent to approximately map
        # the current state to future discounted reward
        # We'll call that target_f
        target_f = model.predict(state)
        target_f[0][action] = target

        # Train the Neural Net with the state and target_f
        model.fit(state, target_f, epochs=1, verbose=0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

In [33]:
def act(state):
    if np.random.rand() <= epsilon:
        # The agent acts randomly
        return env.action_space.sample()
    # Predict the reward value based on the given state
    act_values = model.predict(state)
    # Pick the action based on the predicted reward
    return np.argmax(act_values[0])

In [34]:
for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, 4])
    
    for time_t in range(500):
        env.render()
        
        action = act(state)
        
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, 4])
        
        remember(state, action, reward, next_state, done)
        
        state = next_state
        
        if done:
            print("episode: {}/{}, score: {}".format(e, episodes, time_t))
            
            break
            
    replay(32)

episode: 0/5000, score: 11


ValueError: Sample larger than population or is negative

In [22]:
# Deep Q-learning Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=1000000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.training_frequency = 4
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, total_step, batch_size):
        if len(self.memory) < batch_size:
            return
        if total_step % self.training_frequency == 0:
            minibatch = random.sample(self.memory, batch_size)
            for state, action, reward, next_state, done in minibatch:
                target = reward
                if not done:
                    target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
                target_f = self.model.predict(state)
                target_f[0][action] = target
                self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            self.epsilon = max(self.epsilon, self.epsilon_min)

In [15]:
# initialize gym environment and the agent
env = gym.make('CartPole-v1')
observation_space = env.observation_space.shape[0]
print('Observation space: ', observation_space)
action_space = env.action_space.n
agent = DQNAgent(observation_space, action_space)
episodes = 5000
# Iterate the game
for e in range(episodes):
    # reset state in the beginning of each game
    state = env.reset()
    state = np.reshape(state, [1, observation_space])
    # time_t represents each frame of the game
    # Our goal is to keep the pole upright as long as possible until score of 500
    # the more time_t the more score
    step = 0
    while True:
        step += 1
        # turn this on if you want to render
        # env.render()
        # Decide action
        action = agent.act(state)
        # Advance the game to the next frame based on the action.
        # Reward is 1 for every frame the pole survived
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, observation_space])
        # Remember the previous state, action, reward, and done
        agent.remember(state, action, reward, next_state, done)
        # make next_state the new current state for the next frame.
        state = next_state
        # done becomes True when the game ends
        # ex) The agent drops the pole
        if done:
            # print the score and break out of the loop
            print("episode: {}/{}, score: {}"
                  .format(e, episodes, step))
            break
        # train the agent with the experience of the episode
        agent.replay(32)

Observation space:  4
episode: 0/5000, score: 19
episode: 1/5000, score: 20
episode: 2/5000, score: 11
episode: 3/5000, score: 40


KeyboardInterrupt: 

In [2]:
# Deep Q-learning Agent
class DDQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=1000000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.training_frequency = 4
        self.target_update_frequency = 1000
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.reset_target_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(48, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def reset_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
        
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() < self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, total_step, batch_size):
        if len(self.memory) < batch_size:
            return
        if total_step % self.training_frequency == 0:
            minibatch = random.sample(self.memory, batch_size)
            for state, action, reward, next_state, done in minibatch:
                target = reward
                if not done:
                    target = reward + self.gamma * np.amax(self.target_model.predict(next_state)[0])
                target_f = self.target_model.predict(state)
                target_f[0][action] = target
                self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            self.epsilon = max(self.epsilon, self.epsilon_min)
        if step % self.target_update_frequency == 0:
            self.reset_target_model()

In [42]:
# initialize gym environment and the agent
env = gym.make('Acrobot-v1')
observation_space = env.observation_space.shape[0]
print('Observation space: ', observation_space)
action_space = env.action_space.n
print('Action space: ', action_space)
agent = DDQNAgent(observation_space, action_space)
episodes = 5000
total_step = 0
# Iterate the game
for e in range(episodes):
    # reset state in the beginning of each game
    state = env.reset()
    state = np.reshape(state, [1, observation_space])
    # time_t represents each frame of the game
    # Our goal is to keep the pole upright as long as possible until score of 500
    # the more time_t the more score
    step = 0
    score = 0
    while True:
        step += 1
        total_step += 1
        # turn this on if you want to render
        env.render()
        # Decide action
        action = agent.act(state)
        # Advance the game to the next frame based on the action.
        # Reward is 1 for every frame the pole survived
        next_state, reward, done, _ = env.step(action)
        score += reward
        reward += score / 100 # increase reward as score accumulates
        next_state = np.reshape(next_state, [1, observation_space])
        agent.remember(state, action, reward, next_state, done)
        # train the agent with the experience of the episode
        agent.replay(total_step, 32)
        if done:
            # print the score and break out of the loop
            print("episode: {}/{}, score: {}"
                  .format(e, episodes, reward))
            break
        # make next_state the new current state for the next frame.
        state = next_state

Observation space:  6
Action space:  3
episode: 0/5000, score: -6.0
episode: 1/5000, score: -6.0
episode: 2/5000, score: -6.0
episode: 3/5000, score: -6.0
episode: 4/5000, score: -4.67
episode: 5/5000, score: -6.0
episode: 6/5000, score: -6.0
episode: 7/5000, score: -6.0
episode: 8/5000, score: -6.0
episode: 9/5000, score: -6.0
episode: 10/5000, score: -4.33
episode: 11/5000, score: -6.0
episode: 12/5000, score: -6.0
episode: 13/5000, score: -6.0
episode: 14/5000, score: -3.67
episode: 15/5000, score: -2.15
episode: 16/5000, score: -6.0
episode: 17/5000, score: -4.38
episode: 18/5000, score: -6.0
episode: 19/5000, score: -6.0
episode: 20/5000, score: -6.0
episode: 21/5000, score: -6.0
episode: 22/5000, score: -6.0
episode: 23/5000, score: -6.0
episode: 24/5000, score: -6.0
episode: 25/5000, score: -6.0
episode: 26/5000, score: -6.0
episode: 27/5000, score: -6.0
episode: 28/5000, score: -4.09
episode: 29/5000, score: -1.93
episode: 30/5000, score: -3.54
episode: 31/5000, score: -6.0
epi

KeyboardInterrupt: 

In [6]:
# initialize gym environment and the agent
env = gym.make('LunarLander-v2')
observation_space = env.observation_space.shape[0]
print('Observation space: ', observation_space)
action_space = env.action_space.n
print('Action space: ', action_space)
agent = DDQNAgent(observation_space, action_space)
episodes = 5000
total_step = 0
# Iterate the game
for e in range(episodes):
    # reset state in the beginning of each game
    state = env.reset()
    state = np.reshape(state, [1, observation_space])
    # time_t represents each frame of the game
    # Our goal is to keep the pole upright as long as possible until score of 500
    # the more time_t the more score
    step = 0
    score = 0
    while True:
        step += 1
        total_step += 1
        # turn this on if you want to render
        env.render()
        # Decide action
        action = agent.act(state)
        # Advance the game to the next frame based on the action.
        # Reward is 1 for every frame the pole survived
        next_state, reward, done, _ = env.step(action)
        score += reward
        next_state = np.reshape(next_state, [1, observation_space])
        agent.remember(state, action, reward, next_state, done)
        # train the agent with the experience of the episode
        agent.replay(total_step, 32)
        if done:
            # print the score and break out of the loop
            print("episode: {}/{}, score: {}"
                  .format(e, episodes, score))
            break
        # make next_state the new current state for the next frame.
        state = next_state

Observation space:  8
Action space:  4
episode: 0/5000, score: -336.6009669325832
episode: 1/5000, score: -385.36197845891644
episode: 2/5000, score: -351.3034750668105
episode: 3/5000, score: -123.72327188997868
episode: 4/5000, score: -95.34285632515093
episode: 5/5000, score: -338.09572620348797
episode: 6/5000, score: -762.4843088810056
episode: 7/5000, score: 38.866379114550256
episode: 8/5000, score: -116.71856271629143
episode: 9/5000, score: -141.89130807992095
episode: 10/5000, score: -212.82125611928203
episode: 11/5000, score: -91.80675549085578
episode: 12/5000, score: -110.49760249660756
episode: 13/5000, score: -363.97496912839364
episode: 14/5000, score: -119.29596135808976
episode: 15/5000, score: -564.6885638376685
episode: 16/5000, score: -166.35734838253796
episode: 17/5000, score: -341.841953751224
episode: 18/5000, score: -242.3747193073728
episode: 19/5000, score: -356.01385076245185
episode: 20/5000, score: -445.33345961224063
episode: 21/5000, score: -458.084296

episode: 182/5000, score: 2.1968779947806354
episode: 183/5000, score: -395.54499478811124
episode: 184/5000, score: -300.01604451670164
episode: 185/5000, score: -488.78968552609444
episode: 186/5000, score: -313.8069915451066
episode: 187/5000, score: -161.46554659991244
episode: 188/5000, score: -97.4372896064249
episode: 189/5000, score: -386.0703062349373
episode: 190/5000, score: -199.57258020588992
episode: 191/5000, score: -406.5390394723177
episode: 192/5000, score: -444.2800854637708
episode: 193/5000, score: -236.1841440435443
episode: 194/5000, score: -434.65724655146585
episode: 195/5000, score: -376.50611133071146
episode: 196/5000, score: -379.25526837434796
episode: 197/5000, score: -515.5869998764756
episode: 198/5000, score: -196.47035506724032
episode: 199/5000, score: -307.2988270067148
episode: 200/5000, score: -212.70651601635763
episode: 201/5000, score: -316.02035374067447
episode: 202/5000, score: -260.20069934401033
episode: 203/5000, score: -235.2180144472513

episode: 363/5000, score: -199.75857063331736
episode: 364/5000, score: -287.0444362877013
episode: 365/5000, score: -386.34255243084556
episode: 366/5000, score: -49.37409385212102
episode: 367/5000, score: -146.72888670378956
episode: 368/5000, score: -105.70157325315829
episode: 369/5000, score: -131.7202076025022
episode: 370/5000, score: -466.24837147383835
episode: 371/5000, score: -110.04283682936551
episode: 372/5000, score: -519.9298894726817
episode: 373/5000, score: -325.88547444970067
episode: 374/5000, score: -188.88818369541994
episode: 375/5000, score: -413.68969999307114
episode: 376/5000, score: -457.3269146824696
episode: 377/5000, score: -221.34858445855986
episode: 378/5000, score: -442.94695354628965
episode: 379/5000, score: -333.9039121065436
episode: 380/5000, score: -613.1834088582093
episode: 381/5000, score: -545.7356979674042
episode: 382/5000, score: -511.19227205627664
episode: 383/5000, score: -385.78860682093557
episode: 384/5000, score: -52.156347799591

episode: 544/5000, score: -270.378595438664
episode: 545/5000, score: -415.1669751606499
episode: 546/5000, score: -729.6292575982134
episode: 547/5000, score: -115.59315600817686
episode: 548/5000, score: -212.1602919737211
episode: 549/5000, score: -245.34790261319839
episode: 550/5000, score: -458.9431761085586
episode: 551/5000, score: -422.910050471436
episode: 552/5000, score: -408.24680490072205
episode: 553/5000, score: -328.79587342368404
episode: 554/5000, score: -244.5949583273988
episode: 555/5000, score: -274.89346204557785
episode: 556/5000, score: -188.17749733947537
episode: 557/5000, score: -119.45225562560239
episode: 558/5000, score: -353.2404986337332
episode: 559/5000, score: -242.3788694281758
episode: 560/5000, score: -455.02302947041073
episode: 561/5000, score: -423.58565663031106
episode: 562/5000, score: -264.26680774850377
episode: 563/5000, score: -366.52799509707955
episode: 564/5000, score: -420.79681869805665
episode: 565/5000, score: -259.49574438987327

episode: 725/5000, score: -381.84587752075504
episode: 726/5000, score: -365.6155321748407
episode: 727/5000, score: -212.3887930556865
episode: 728/5000, score: -182.93091373543223
episode: 729/5000, score: -169.26193910187868
episode: 730/5000, score: -307.9056581184158
episode: 731/5000, score: -556.7590298586622
episode: 732/5000, score: -277.971771428906
episode: 733/5000, score: -381.71442391629324
episode: 734/5000, score: -282.4986344684015
episode: 735/5000, score: -125.59613787476553
episode: 736/5000, score: -37.441580330835706
episode: 737/5000, score: -249.8251975250735
episode: 738/5000, score: -330.62451246307637
episode: 739/5000, score: -95.13247633215552
episode: 740/5000, score: -248.80034077894808
episode: 741/5000, score: -308.8446346081027
episode: 742/5000, score: -396.29341827581806
episode: 743/5000, score: -183.12306650142204
episode: 744/5000, score: -353.54435863156846
episode: 745/5000, score: -500.2094050869641
episode: 746/5000, score: -392.80032509532333

...

episode: 1085/5000, score: -205.10216427598974
episode: 1086/5000, score: -300.5902517265795
episode: 1087/5000, score: -360.8319908793471
episode: 1088/5000, score: -179.6991373652637
episode: 1089/5000, score: -152.26179580756883
episode: 1090/5000, score: -358.9988030384427
episode: 1091/5000, score: -78.50150596068151
episode: 1092/5000, score: -389.09468474180676
episode: 1093/5000, score: -422.2875221201664
episode: 1094/5000, score: -447.97250996036274
episode: 1095/5000, score: -394.8531246865094
episode: 1096/5000, score: -474.5643714767552
episode: 1097/5000, score: -400.00205620922594
episode: 1098/5000, score: -464.62464937458003
episode: 1099/5000, score: -440.6826051041051
episode: 1100/5000, score: -378.76668475046125
episode: 1101/5000, score: -459.3206948013461
episode: 1102/5000, score: -355.9027451532726
episode: 1103/5000, score: -416.5244421361186
episode: 1104/5000, score: -419.5598116161777
episode: 1105/5000, score: -428.64439787135274
episode: 1106/5000, score:

episode: 1262/5000, score: -182.57842641681864
episode: 1263/5000, score: -231.19125811864274
episode: 1264/5000, score: -539.9101529329756
episode: 1265/5000, score: -282.2666385011238
episode: 1266/5000, score: -289.8926314755585
episode: 1267/5000, score: -42.89350709128806
episode: 1268/5000, score: -252.53145571296218
episode: 1269/5000, score: -13.897698911269643
episode: 1270/5000, score: -270.09236156113656
episode: 1271/5000, score: -292.81962486929
episode: 1272/5000, score: -168.0147783606914
episode: 1273/5000, score: -94.42005847230112
episode: 1274/5000, score: -211.2519927989751
episode: 1275/5000, score: -102.16734368460193
episode: 1276/5000, score: -522.7324638438224
episode: 1277/5000, score: -220.93101886750512
episode: 1278/5000, score: -205.48868002231325
episode: 1279/5000, score: -230.93947849583333
episode: 1280/5000, score: -194.62772144214668
episode: 1281/5000, score: -175.15412331521043
episode: 1282/5000, score: -229.32310526641
episode: 1283/5000, score: 

episode: 1438/5000, score: -143.3311748300194
episode: 1439/5000, score: -160.3189634355689
episode: 1440/5000, score: -157.94138522326062
episode: 1441/5000, score: -139.19942350928056
episode: 1442/5000, score: -86.10915193499804
episode: 1443/5000, score: -132.20450516110057
episode: 1444/5000, score: -129.4119678093312
episode: 1445/5000, score: -135.03157728160724
episode: 1446/5000, score: -154.36862826508855
episode: 1447/5000, score: -106.09800540949277
episode: 1448/5000, score: -86.89588172153634
episode: 1449/5000, score: -95.03683153182605
episode: 1450/5000, score: -42.12651023696499
episode: 1451/5000, score: 190.72566910800106
episode: 1452/5000, score: -12.937931558777308
episode: 1453/5000, score: -55.33945380177133
episode: 1454/5000, score: -36.522941762829895
episode: 1455/5000, score: -48.686589444434546
episode: 1456/5000, score: 12.355596854658657
episode: 1457/5000, score: -33.269787614310914
episode: 1458/5000, score: -133.9959968724559
episode: 1459/5000, scor

episode: 1614/5000, score: -172.48351784236954
episode: 1615/5000, score: -163.94880963400576
episode: 1616/5000, score: -147.47010470008547
episode: 1617/5000, score: -109.01189924963296
episode: 1618/5000, score: -163.64759315511503
episode: 1619/5000, score: -142.92804399248544
episode: 1620/5000, score: -181.0654874411369
episode: 1621/5000, score: -148.52093733298605
episode: 1622/5000, score: -164.36487524162115
episode: 1623/5000, score: -127.94188613944989
episode: 1624/5000, score: -172.42616785874677
episode: 1625/5000, score: -134.02685522338948
episode: 1626/5000, score: -136.20084600818393
episode: 1627/5000, score: -155.93928729595535
episode: 1628/5000, score: -103.17359263963033
episode: 1629/5000, score: -137.60913297245077
episode: 1630/5000, score: -153.55591036387142
episode: 1631/5000, score: -156.30152962201146
episode: 1632/5000, score: -143.54224207607203
episode: 1633/5000, score: -126.12156311288985
episode: 1634/5000, score: -132.38501203139074
episode: 1635/

KeyboardInterrupt: 

In [7]:
env.close()

In [52]:
# Deep Q-learning Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=1000000)
        self.gamma = 0.99    # discount rate
        self.rho = 0.95
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        frame_input = Input(self.state_size)
        action_input = Input((self.action_size,))
        normalized = Lambda(lambda x: x / 255.0)(frame_input)
        conv_1 = Convolution2D(16, (8, 8), strides=(4, 4), activation='relu')(normalized)
        conv_2 = Convolution2D(32, (4, 4), strides=(2, 2), activation='relu')(conv_1)
        conv_flattened = Flatten()(conv_2)
        hidden = Dense(256, activation='relu')(conv_flattened)
        output = Dense(self.action_size)(hidden)
        filtered_output = Multiply()([output, action_input])
        model = Model(inputs=[frame_input, action_input], outputs=filtered_output)
        optimizer = RMSprop(lr=self.learning_rate, rho=self.rho, epsilon=self.epsilon)
        model.compile(loss='mse', optimizer=optimizer)
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [21]:
# Deep Q-learning Agent
class DDQNAgent:
    def __init__(self, game, state_size, action_size):
        self.game = game
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=1000000)
        self.gamma = 0.99    # discount rate
        self.rho = 0.95
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.000001059
        self.learning_rate = 0.00025
        self.training_step = 4
        self.target_update_frequency = 40000
        self.save_frequency = 30000
        self.replay_start_size = 50000
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.reset_target_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Convolution2D(32, 8, input_shape=self.state_size, strides=(4, 4), activation='relu'))
        model.add(Convolution2D(64, 4, input_shape=self.state_size, strides=(2, 2), activation='relu'))
        model.add(Convolution2D(64, 3, input_shape=self.state_size, strides=(1, 1), activation='relu'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(self.action_size))
        optimizer = RMSprop(lr=self.learning_rate, rho=self.rho, epsilon=0.01)
        model.compile(loss='mse', optimizer=optimizer)
        return model

    def save_model(self, step):
        print('Saving model: atari' + str(self.game) + str(step) + '.h5')
        self.model.save('atari' + str(self.game) + str(step) + '.h5')
        
    def reset_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon or len(self.memory) < self.replay_start_size:
            return random.randrange(self.action_size)
        act_values = self.model.predict(np.expand_dims(np.asarray(state), axis=0))
        return np.argmax(act_values[0])  # returns action

    def replay(self, step, batch_size):
        if len(self.memory) < self.replay_start_size:
            return
        if step % self.training_step == 0:
            minibatch = random.sample(self.memory, batch_size)
            for state, action, reward, next_state, done in minibatch:
                state = np.expand_dims(np.asarray(state), axis=0)
                next_state = np.expand_dims(np.asarray(next_state), axis=0)
                target = reward
                if not done:
                    target = reward + self.gamma * np.amax(self.target_model.predict(next_state)[0])
                target_f = self.model.predict(state)
                target_f[0][action] = target
                self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon -= self.epsilon_decay
            self.epsilon = max(self.epsilon_min, self.epsilon)
        if step % self.target_update_frequency == 0:
            self.reset_target_model()
        if step % self.save_frequency == 0:
            self.save_model(step)

In [4]:
def to_grayscale(img):
    return np.mean(img, axis=2).astype(np.uint8)

def downsample(img):
    return img[::2, ::2]

def preprocess(img):
    return np.expand_dims(to_grayscale(downsample(img)), axis=2)

def transform_reward(reward):
    return np.sign(reward)

In [ ]:
# initialize gym environment and the agent
game = 'Pong'
env = wrap_deepmind(gym.make(game + 'Deterministic-v4'), frame_stack=True)
observation_space = env.observation_space.shape
action_space = env.action_space.n
print('Observation space: ', observation_space)
print('Action space: ', action_space)
agent = DDQNAgent(game, observation_space, action_space)
episodes = 100000
total_step = 0
# Iterate the game
for e in range(episodes):
    # reset state in the beginning of each game
    state = env.reset()
    # time_t represents each frame of the game
    # Our goal is to keep the pole upright as long as possible until score of 500
    # the more time_t the more score
    step = 0
    score = 0
    while True:
        total_step += 1
        step += 1
        # turn this on if you want to render
        #env.render()
        # Decide action
        action = agent.act(state)
        # Advance the game to the next frame based on the action.
        # Reward is 1 for every frame the pole survived
        next_state, reward, done, _ = env.step(action)
        score += reward
        # Remember the previous state, action, reward, and done
        agent.remember(state, action, reward, next_state, done)
        # make next_state the new current state for the next frame.
        state = next_state
        # train the agent with the experience of the episode
        agent.replay(total_step, 32)
        # done becomes True when the game ends
        # ex) The agent drops the pole
        if done:
            # print the score and break out of the loop
            print("episode: {}/{}, steps: {}, total steps: {}, score: {}"
                  .format(e, episodes, step, total_step, score))
            break


Observation space:  (84, 84, 4)
Action space:  6
episode: 0/100000, steps: 850, total steps: 850, score: -21.0
episode: 1/100000, steps: 762, total steps: 1612, score: -21.0
episode: 2/100000, steps: 1012, total steps: 2624, score: -19.0
episode: 3/100000, steps: 942, total steps: 3566, score: -21.0
episode: 4/100000, steps: 878, total steps: 4444, score: -20.0
episode: 5/100000, steps: 1002, total steps: 5446, score: -21.0
episode: 6/100000, steps: 1146, total steps: 6592, score: -20.0
episode: 7/100000, steps: 896, total steps: 7488, score: -20.0
episode: 8/100000, steps: 997, total steps: 8485, score: -18.0
episode: 9/100000, steps: 976, total steps: 9461, score: -20.0
episode: 10/100000, steps: 850, total steps: 10311, score: -21.0
episode: 11/100000, steps: 822, total steps: 11133, score: -21.0
episode: 12/100000, steps: 836, total steps: 11969, score: -20.0
episode: 13/100000, steps: 790, total steps: 12759, score: -21.0
episode: 14/100000, steps: 991, total steps: 13750, score: 

episode: 124/100000, steps: 1425, total steps: 114939, score: -15.0
episode: 125/100000, steps: 809, total steps: 115748, score: -21.0
episode: 126/100000, steps: 896, total steps: 116644, score: -20.0
episode: 127/100000, steps: 989, total steps: 117633, score: -20.0
episode: 128/100000, steps: 961, total steps: 118594, score: -20.0
episode: 129/100000, steps: 822, total steps: 119416, score: -21.0
Saving model: atariPong120000.h5
episode: 130/100000, steps: 880, total steps: 120296, score: -21.0
episode: 131/100000, steps: 1006, total steps: 121302, score: -20.0
episode: 132/100000, steps: 822, total steps: 122124, score: -21.0
episode: 133/100000, steps: 979, total steps: 123103, score: -20.0
episode: 134/100000, steps: 884, total steps: 123987, score: -21.0
episode: 135/100000, steps: 868, total steps: 124855, score: -20.0
episode: 136/100000, steps: 868, total steps: 125723, score: -20.0
episode: 137/100000, steps: 970, total steps: 126693, score: -21.0
episode: 138/100000, steps:

episode: 244/100000, steps: 866, total steps: 224930, score: -21.0
episode: 245/100000, steps: 781, total steps: 225711, score: -21.0
episode: 246/100000, steps: 912, total steps: 226623, score: -21.0
episode: 247/100000, steps: 762, total steps: 227385, score: -21.0
episode: 248/100000, steps: 790, total steps: 228175, score: -21.0
episode: 249/100000, steps: 973, total steps: 229148, score: -20.0
episode: 250/100000, steps: 824, total steps: 229972, score: -21.0
episode: 251/100000, steps: 762, total steps: 230734, score: -21.0
episode: 252/100000, steps: 824, total steps: 231558, score: -21.0
episode: 253/100000, steps: 877, total steps: 232435, score: -20.0
episode: 254/100000, steps: 809, total steps: 233244, score: -21.0
episode: 255/100000, steps: 1034, total steps: 234278, score: -20.0
episode: 256/100000, steps: 988, total steps: 235266, score: -20.0
episode: 257/100000, steps: 822, total steps: 236088, score: -21.0
episode: 258/100000, steps: 851, total steps: 236939, score: 

episode: 365/100000, steps: 822, total steps: 334169, score: -21.0
episode: 366/100000, steps: 1043, total steps: 335212, score: -19.0
episode: 367/100000, steps: 889, total steps: 336101, score: -20.0
episode: 368/100000, steps: 972, total steps: 337073, score: -20.0
episode: 369/100000, steps: 1043, total steps: 338116, score: -19.0
episode: 370/100000, steps: 841, total steps: 338957, score: -21.0
episode: 371/100000, steps: 947, total steps: 339904, score: -21.0
episode: 372/100000, steps: 843, total steps: 340747, score: -21.0
episode: 373/100000, steps: 842, total steps: 341589, score: -21.0
episode: 374/100000, steps: 1198, total steps: 342787, score: -18.0
episode: 375/100000, steps: 1149, total steps: 343936, score: -19.0
episode: 376/100000, steps: 841, total steps: 344777, score: -20.0
episode: 377/100000, steps: 1010, total steps: 345787, score: -18.0
episode: 378/100000, steps: 840, total steps: 346627, score: -20.0
episode: 379/100000, steps: 818, total steps: 347445, sco

episode: 486/100000, steps: 790, total steps: 440493, score: -21.0
episode: 487/100000, steps: 878, total steps: 441371, score: -21.0
episode: 488/100000, steps: 824, total steps: 442195, score: -21.0
episode: 489/100000, steps: 882, total steps: 443077, score: -21.0
episode: 490/100000, steps: 824, total steps: 443901, score: -21.0
episode: 491/100000, steps: 762, total steps: 444663, score: -21.0
episode: 492/100000, steps: 850, total steps: 445513, score: -21.0
episode: 493/100000, steps: 840, total steps: 446353, score: -20.0
episode: 494/100000, steps: 762, total steps: 447115, score: -21.0
episode: 495/100000, steps: 822, total steps: 447937, score: -21.0
episode: 496/100000, steps: 921, total steps: 448858, score: -20.0
episode: 497/100000, steps: 1025, total steps: 449883, score: -19.0
Saving model: atariPong450000.h5
episode: 498/100000, steps: 762, total steps: 450645, score: -21.0
episode: 499/100000, steps: 840, total steps: 451485, score: -21.0
episode: 500/100000, steps: 

In [4]:
env.close()

In [12]:
from gym.utils.play import play
%matplotlib inline
play(wrap_deepmind(gym.make('PongDeterministic-v4'), frame_stack=True, scale=True))
state = env.reset()
print(env2.observation_space.shape)
plt.imshow(state)
env2.close()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


NameError: name 'env2' is not defined